In [12]:
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
import copy
import os
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import cv2
import sys
import time

In [3]:
#Label file:
data_path = '/home/iiitd/crime'
classes = os.listdir(data_path)
decoder = {}
for i in range(len(classes)):
    decoder[classes[i]] = i
encoder = {}
for i in range(len(classes)):
    encoder[i] = classes[i]

In [4]:
id = list()
path = '/home/iiitd/crime'
for i in os.listdir(path):
  p1 = os.path.join(path,i)
  for j in os.listdir(p1):
    p2 = os.path.join(p1,j)
    id.append((i,p2))


In [5]:
class video_dataset(Dataset):
    def __init__(self,frame_list,sequence_length = 16,transform = None):
        self.frame_list = frame_list
        self.transform = transform
        self.sequence_length = sequence_length
    def __len__(self):
        return len(self.frame_list)
    def __getitem__(self,idx):
        label,path = self.frame_list[idx]
       
        img = cv2.imread(path)
        seq_img = list()
        for i in range(16):
          img1 = img[:,128*i:128*(i+1),:]
          if(self.transform):
            img1 = self.transform(img1)
          seq_img.append(img1)
        seq_image = torch.stack(seq_img)
        seq_image = seq_image.reshape(3,16,im_size,im_size)
#         print(label,path)
        return seq_image,label,path

In [6]:
im_size = 128
mean = [0.4889, 0.4887, 0.4891]
std = [0.2074, 0.2074, 0.2074]


train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.RandomRotation(degrees=10),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])

train_data = video_dataset(id,sequence_length = 16,transform = train_transforms)
train_loader = DataLoader(train_data,batch_size = 8,num_workers = 4 ,shuffle = True)
test_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor()])

test_data = video_dataset(id,sequence_length = 16,transform = test_transforms)
test_loader = DataLoader(test_data,batch_size = 1,num_workers = 4 ,shuffle = True)
dataloaders = {'train':train_loader, 'test':test_loader}

In [7]:
from model import resnet50
model = resnet50(class_num=8)
model.load_state_dict(torch.load('/home/iiitd/weights_crime/c3d_19.h5'))

<All keys matched successfully>

In [13]:
from torch.autograd import Variable
for batch_i, (X, y,p) in enumerate(dataloaders['test']):
    a=time.time()
    image_sequences = Variable(X)
    labels = y
    predictions = model(image_sequences)
    pred_label=str(encoder[predictions.detach().argmax(1).item()])
    true_label=labels[0]
    
#     if pred_label==true_label:
    p=p[0]
    ind1=p.rfind('/')
    ind2=p.rfind('_')
    VID=p[ind1+1:ind2]+'_x264.mp4'

    print('prediction: ',encoder[predictions.detach().argmax(1).item()],' label: ',labels[0],' video: ', VID)
    b=time.time()
    print(b-a)



prediction:  Burglary  label:  Assault  video:  Assault008_x264.mp4
2.033311605453491
prediction:  Burglary  label:  normal  video:  Normal_Videos_913_x264.mp4
2.4172916412353516
prediction:  Burglary  label:  Abuse  video:  Abuse038_x264.mp4
0.937291145324707
prediction:  normal  label:  Arrest  video:  Arrest016_x264.mp4
0.9206523895263672
prediction:  Burglary  label:  normal  video:  Normal_Videos_903_x264.mp4
1.6439876556396484
prediction:  Burglary  label:  Arson  video:  Arson019_x264.mp4
0.8921153545379639
prediction:  Burglary  label:  Fighting  video:  Fighting035_x264.mp4
0.8476724624633789
prediction:  Burglary  label:  Assault  video:  Assault019_x264.mp4
0.9133553504943848
prediction:  Burglary  label:  normal  video:  Normal_Videos_925_x264.mp4
0.8897860050201416
prediction:  Burglary  label:  normal  video:  Normal_Videos_059_x264.mp4
0.8747642040252686
prediction:  Burglary  label:  Assault  video:  Assault049_x264.mp4
0.8524360656738281
prediction:  Burglary  label:  

KeyboardInterrupt: 

In [7]:
from clr import *
device = 'cuda'
cls_criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum = 0.9,weight_decay = 1e-4)
num_epochs = 20
onecyc = OneCycle(len(train_loader)*num_epochs,1e-3)

In [ ]:
os.makedirs('/home/iiitd/weights_crime',exist_ok = True)
from torch.autograd import Variable
iteration = 0
acc_all = list()
loss_all = list()
    
for epoch in range(num_epochs):
    print('')
    print(f"--- Epoch {epoch} ---")
    phase1 = dataloaders.keys()
    for phase in phase1:
        print('')
        print(f"--- Phase {phase} ---")
        epoch_metrics = {"loss": [], "acc": []}
        for batch_i, (X, y) in enumerate(dataloaders[phase]):
            #iteration = iteration+1
            image_sequences = Variable(X, requires_grad=True)
            labels = Variable(y, requires_grad=False)
            optimizer.zero_grad()
            #model.lstm.reset_hidden_state()
            predictions = model(image_sequences)
            loss = cls_criterion(predictions, labels)
            acc = 100 * (predictions.detach().argmax(1) == labels).cpu().numpy().mean()
            loss.backward()
            optimizer.step()
            epoch_metrics["loss"].append(loss.item())
            epoch_metrics["acc"].append(acc)
            if(phase=='train'):
                lr,mom = onecyc.calc()
                update_lr(optimizer, lr)
                update_mom(optimizer, mom)
            batches_done = epoch * len(dataloaders[phase]) + batch_i
            batches_left = num_epochs * len(dataloaders[phase]) - batches_done
            sys.stdout.write(
                    "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f), Acc: %.2f%% (%.2f%%)]"
                    % (
                        epoch,
                        num_epochs,
                        batch_i,
                        len(dataloaders[phase]),
                        loss.item(),
                        np.mean(epoch_metrics["loss"]),
                        acc,
                        np.mean(epoch_metrics["acc"]),
                    )
                )

                # Empty cache
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
        print('')
        print('{} , acc: {}'.format(phase,np.mean(epoch_metrics["acc"])))
        torch.save(model.state_dict(),'/home/iiitd/weights_crime/c3d_{}.h5'.format(epoch))
        if(phase=='train'):
          acc_all.append(np.mean(epoch_metrics["acc"]))
          loss_all.append(np.mean(epoch_metrics["loss"]))


--- Epoch 0 ---

--- Phase train ---
[Epoch 0/20] [Batch 112/688] [Loss: 2.185052 (2.246415), Acc: 25.00% (20.91%)]

In [ ]:
def error_plot(loss):
    plt.figure(figsize=(10,5))
    plt.plot(loss)
    plt.title("Training loss plot")
    plt.xlabel("epochs")
    plt.ylabel("Loss")
    plt.show()
def acc_plot(acc):
    plt.figure(figsize=(10,5))
    plt.plot(acc)
    plt.title("Training accuracy plot")
    plt.xlabel("epochs")
    plt.ylabel("accuracy")
    plt.show()

In [ ]:
loss_all

In [ ]:
error_plot(loss_all)

In [ ]:
acc_plot(acc_all)